# TinyMLOps with Arduino

In [ ]:
# default_exp train

# Prepare data 

## Prepare bucket

In [ ]:
#hide
import os
import boto3

mlflow_bucket_name="mlflow"
s3_client = boto3.client('s3', endpoint_url=os.environ['MLFLOW_S3_ENDPOINT_URL'])
s3_client.create_bucket(Bucket=mlflow_bucket_name)

In [ ]:
#export
import os
import pandas as pd
import warnings
import sys
import numpy as np
import joblib
import tensorflow as tf
import fileinput
import sklearn
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

## Train and load to mlflow  

### Train, Save and Evaluate 

In [ ]:
#export

classes = []

for file in os.listdir('./data'):
    if file.endswith(".csv"):
        classes.append(os.path.splitext(file)[0])

classes.sort()

num_classes = len(classes)

input_list = []
output_list = []


e=np.eye(3)
for index in range(num_classes):
    class_file = classes[index]
    df = pd.read_csv(f"./data/{class_file}.csv")
    input_list.append(df)
    #output_list.append(np.full((df.shape[0],1),index))
    output_list.append(np.full((df.shape[0],num_classes),e[index]))
    
    
X = pd.concat(input_list, axis=0).to_numpy()
y = np.vstack(output_list)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

In [ ]:
#export
model = tf.keras.Sequential([
  layers.Dense(8, activation='relu'),
  layers.Dense(5, activation='relu'),
  layers.Dense(num_classes, activation='softmax'),
])

model.compile(optimizer='adam', loss='mse', metrics=['mae','accuracy'])
history = model.fit(X_train, y_train, epochs=400, batch_size=4, validation_data=(X_val, y_val))

In [ ]:
#export
loss, mae, accuracy = model.evaluate(X_test, y_test)
print("Accuracy", accuracy)
print("MAE", mae)

In [ ]:
#export
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('model.tflite','wb') as f:
    f.write(tflite_model)

with open('model.h','w') as f:
    f.write('const unsigned char model[] = {\n')
    
    size=12
    l=int(len(tflite_model) / 12)
    if len(tflite_model) % 12 != 0:
        l=l+1

    for i in range(l):
        end="\n" if i == l-1 else ",\n"
        f.write('  '+', '.join([f'0x{hex(i)[2:].rjust(2, "0")}' for i in tflite_model[i*size:i*size+size]])+end) 
    f.write('};\n')
    
arduino_env = [
    "Arduino_TensorFlowLite@2.4.0-ALPHA",
    "Arduino_APDS9960@1.0.3",
    "Arduino_HTS221@1.0.0",
    "Arduino_LPS22HB@1.0.1",
    "Arduino_LSM9DS1@1.1.0",
    "arduinoFFT@1.5.6"
]

with open('requirements.ino.txt','w') as f:
    for env in arduino_env:
        f.write(f"{env}\n")
        
artifacts = {
    "tfmodel": 'model.tflite',
    "model_header": "model.h",
    "requirements.ino.txt": "requirements.ino.txt",
    "artifacts": "artifacts.ino"
}

### Custom MLflow model wrapper

In [ ]:
#export 
import mlflow.pyfunc
class ArduinoWrapper(mlflow.pyfunc.PythonModel):
    def load_context(self, context): ...
    def predict(self, context, model_input): ...

### Conda environment 

In [ ]:
#export 
from sys import version_info
import sklearn
import cloudpickle

PYTHON_VERSION = f"{version_info.major}.{version_info.minor}.{version_info.micro}"

conda_env = {
    'channels': ['defaults'],
    'dependencies': [
        f'python={PYTHON_VERSION}',
        'pip',
        {
            'pip':[
                'mlflow',
                f'scikit-learn=={sklearn.__version__}',
                f'tensorflow=={tf.__version__}'
            ]
        }
    ],
    'name': 'serving_propensity_model'
}

In [ ]:
#export
import os
os.environ['MLFLOW_TRACKING_URI']='http://mlflow:5000'

import warnings
import sys

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import mlflow.pyfunc

#conda_env=mlflow.pyfunc.get_default_conda_env()

with mlflow.start_run():
    
    #mlflow.log_param("var_smoothing", input_params['var_smoothing'])
    mlflow.log_metric("Accuracy", accuracy)
    mlflow.log_metric("MAE", mae)
    
    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

    if tracking_url_type_store != "file":
        mlflow.pyfunc.log_model("model",
                                 registered_model_name="arduino_model",
                                 python_model=ArduinoWrapper(),
                                 artifacts=artifacts,
                                 conda_env=conda_env)
    else:
        mlflow.pyfunc.log_model("model",
                                 path=my_model_path,
                                 python_model=ArduinoWrapper(),
                                 artifacts=artifacts,
                                 conda_env=conda_env)

## Export train code 

The above code will be exported to the python file using nbdev library (export, hide, default_exp keyworkd are needed ) 

In [31]:
#hide
from nbdev.export import *
notebook2script()

Converted mlflow_arduino.ipynb.


## Train from command using mlflow

In [32]:
!mlflow run . --no-conda --experiment-name="arduino"

2021/07/09 21:30:35 INFO mlflow.projects.utils: === Created directory /tmp/tmp0dn9cf_c for downloading remote URIs passed to arguments of type 'path' ===
2021/07/09 21:30:35 INFO mlflow.projects.backend.local: === Running command 'python3 ./arduino/train.py' in run with ID 'b36cb97f795a44848e172186e260caf8' === 
2021-07-09 21:30:36.246074: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-07-09 21:30:36.246147: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-07-09 21:30:38.653984: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-07-09 21:30:38.654054: W tensorflow/stream_executor/cuda/cuda_driver.cc:3